In [ ]:
import os
import uuid
from typing import List
from evolutionary.evolution_base import SolutionCandidate


def save_images_from_generation(population: List[SolutionCandidate], generation: int):
    # Create a directory for the current generation
    generation_dir = f"generation_{generation}"
    os.makedirs(generation_dir, exist_ok=True)

    for candidate in population:
        image_solution_data = candidate.result  

        # Save each image in the solution
        for image in image_solution_data.images:
            image_name = f"{uuid.uuid4()}.png"  # Unique image name
            image_path = os.path.join(generation_dir, image_name)
            image.save(image_path)

In [4]:
from model_helpers.auto_pipeline import auto_diffusion_pipeline
pipe = auto_diffusion_pipeline("stabilityai/sdxl-turbo")

In [6]:
from evolutionary.prompt_encoding.image_creation import SDXLPromptEncodingImageCreator
from evolutionary.prompt_encoding.variation import PooledArithmeticCrossover, PooledUniformGaussianMutator, UniformGaussianMutatorArguments
from evolutionary.selectors import TournamentSelector
from evolutionary.algorithms.simple_genetic import GeneticAlgorithm
from evolutionary.image_evaluators import AestheticsImageEvaluator  

population_size = 10
num_generations = 50  

# Create the necessary components for the genetic algorithm
creator = SDXLPromptEncodingImageCreator(pipeline=pipe, batch_size=2, inference_steps=3)
evaluator = AestheticsImageEvaluator()  
crossover = PooledArithmeticCrossover(crossover_rate=0.5, crossover_rate_pooled=0.5)
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.1,  mutation_strength=0.5, clamp_range=(-20, 20))
mutator = PooledUniformGaussianMutator(mutation_arguments, mutation_arguments)
selector = TournamentSelector(tournament_size=3)

# Prepare initial arguments
init_args = [creator.arguments_from_prompt(f"test cat {i}") for i in range(population_size)]

# Create and run the genetic algorithm
ga = GeneticAlgorithm(
    solution_creator=creator,
    evaluator=evaluator,
    mutator=mutator,
    crossover=crossover,
    selector=selector,
    population_size=population_size,
    num_generations=num_generations,
    initial_arguments=init_args
)

best_solution = ga.run()

Keyword arguments {'safety_checker': None, 'requires_safety_checker': False} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loaded StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.25.0",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}


  0%|          | 0/3 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for +: 'PooledPromptEmbedData' and 'str'